# <font color=#025dfa> Papatya, Karahindiba, Gül, Ayçiçeği ve Lale Adlı çiçekleri tahmin Eden Model - Model Predicting Flowers Named Daisy, Dandelion, Rose, Sunflower and Tulip

TR = Her yorum satırı kendisini üstündeki koda aittir. İlk olarak Türkçe, son olarak İngilizce yazıldı.

EN = Each comment line belongs to the code above it. It was first written in Turkish and lastly in English.

TR = Papatya, karahindiba, gül, ayçiçeği ve lale gibi çiçek türlerini görüntüler üzerinden otomatik olarak tanıyan bir model geliştirmeyi amaçlayan bu proje, derin öğrenme ve görüntü işleme teknikleri kullanılarak çiçek türlerinin sınıflandırılmasını hedeflemektedir.

EN = This project aims to develop a model that automatically recognizes flower types such as daisies, dandelions, roses, sunflowers and tulips from images and aims to classify flower types using deep learning and image processing techniques.

Kaynak/Source = https://thecleverprogrammer.com/2020/11/24/flower-recognition-with-python/

In [1]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Conv2D, InputLayer , Reshape , MaxPooling2D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import VGG16,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Tek satırda resimleri okumayı sağlıyor

from PIL import Image

In [2]:
from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [7]:
labels = ['daisy','dandelion','rose','sunflower','tulip']
# TR = Etiketleri saklayacak bir liste oluşturur.
# EN = Creates a list to store the labels.

img_path = img_path = '/content/drive/My Drive/flowers/'
# TR = Görüntü dosyalarının bulunduğu dizini veya yolunu saklayacak bir değişken oluşturur.
# EN = Creates a variable to store the directory or path where the image files are located.


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \UXXXXXXXX escape (3235128216.py, line 5)

In [4]:
img_list = []
label_list = []

for label in labels:
    label_path = os.path.join(img_path, label)  # TR: Etiketin dizin yolunu oluşturur. / EN: Creates the directory path for the label.

    if os.path.exists(label_path):  # TR: Klasörün var olup olmadığını kontrol eder. / EN: Checks if the folder exists.
        for img_file in os.listdir(label_path):
            img_list.append(os.path.join(label_path, img_file))
            label_list.append(label)
    else:
        print(f"Klasör bulunamadı: {label_path}")  # TR: Klasör bulunamadı mesajı verir. / EN: Prints folder not found message.


Klasör bulunamadı: /content/drive/My Drive/flowers/daisy
Klasör bulunamadı: /content/drive/My Drive/flowers/dandelion
Klasör bulunamadı: /content/drive/My Drive/flowers/rose
Klasör bulunamadı: /content/drive/My Drive/flowers/sunflower
Klasör bulunamadı: /content/drive/My Drive/flowers/tulip


In [5]:
df = pd.DataFrame({'img': img_list, 'label': label_list})
# TR = Görüntü dosyası yollarını ve etiketlerini içeren yeni bir DataFrame oluşturur.
# EN = Creates a new DataFrame containing the image file paths and their corresponding labels.

In [ ]:
d = {'daisy':0,'dandelion':1,'rose':2,'sunflower':3,'tulip':4}
# TR = Değer içeren bir sözlük oluşturur.
# EN = Creates a dictionary containing values.

In [ ]:
df['encode_label'] = df['label'].map(d)
# TR = 'label' sütunundaki etiketleri, 'd' sözlüğündeki karşılıklarıyla değiştirir ve sonuçları 'encode_label' adlı yeni bir sütuna ekler.
# EN = Maps the labels in the 'label' column to their corresponding values in the dictionary 'd' and adds the results to a new column named 'encode_label'.

## <font color=#FFD700> EDA Keşif Amaçlı Veri Analizi - EDA - Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
for label in labels:  
    # TR = Her etiket için döngü başlatılır  
    # EN = Start loop for each label
    
    label_images = df[df['label'] == label]['img'].tolist()  
    # TR = İlgili etiketle eşleşen tüm görüntü yolları bir listeye alınır  
    # EN = All image paths matching the label are retrieved into a list

    if len(label_images) >= 9:  
        # TR = Eğer görüntü sayısı 9 veya daha fazlaysa işlem devam eder  
        # EN = Proceed if there are 9 or more images

        selected_images = random.sample(label_images, 9)  
        # TR = Görüntüler arasından rastgele 9 tanesi seçilir  
        # EN = Randomly select 9 images from the list
        
        plt.figure(figsize=(15, 15))  
        # TR = 15x15 boyutlarında bir figür oluşturulur  
        # EN = Create a figure of size 15x15
        
        for i, img_path in enumerate(selected_images):  
            # TR = Seçilen her bir görüntü için döngü başlatılır  
            # EN = Loop through each selected image
            
            img = mpimg.imread(img_path)  
            # TR = Görüntü dosyası okunur  
            # EN = Read the image file
            
            plt.subplot(3, 3, i + 1)  
            # TR = 3x3'lük bir yerleşimde görüntü konumlandırılır  
            # EN = Place the image in a 3x3 grid
            
            plt.imshow(img)  
            # TR = Görüntü ekranda gösterilir  
            # EN = Display the image
            
            plt.title(label)  
            # TR = Her görüntüye ait başlık olarak etiket eklenir  
            # EN = Set the label as the title for each image
            
            plt.axis('off')  
            # TR = Eksenler gizlenir  
            # EN = Hide the axes
        
        plt.show()  
        # TR = Görüntüler ekranda gösterilir  
        # EN = Display the images on the screen
        
    else:  
        # TR = Eğer yeterli görüntü yoksa uyarı verir  
        # EN = If not enough images are available, show a warning message
        
        print(f"Yeterli görüntü bulunamadı/Not enough images found: {label}")  
        # TR = Ekranda yeterli görüntü bulunamadığını belirtir  
        # EN = Print a message that not enough images were found


In [ ]:
x = []
# TR = Görüntüleri saklayacak bir liste oluşturur.
# EN = Creates a list to store the images.

size = 64
# TR = Görüntülerin yeniden boyutlandırılacağı hedef boyutu belirler.
# EN = Sets the target size to which images will be resized.

for img in df['img']:
    img = cv2.imread(str(img))
    # TR = Görüntüyü dosyadan okur. 'str(img)' dosya yolunu belirtir.
    # EN = Reads the image from file. 'str(img)' specifies the file path.

    img = cv2.resize(img, (size, size))
    # TR = Görüntüyü belirtilen boyuta yeniden boyutlandırır, burada her iki boyut size olarak belittik.
    # EN = Resizes the image to the specified size, where we specify both dimensions as size.

    img = img / 255.0
    # TR = Görüntüyü normalleştirir, pixel değerlerini 0 ile 1 arasına çeker.
    # EN = Normalizes the image, scaling pixel values to be between 0 and 1.

    x.append(img)
    # TR = Normalleştirilmiş ve yeniden boyutlandırılmış görüntüyü listeye ekler.
    # EN = Appends the normalized and resized image to the list.

In [ ]:
x=np.array(x)
# TR = Yazıyı resim olarak okuyor bu yüzden diziye çevirdik.
# EN = The text is read as an image, so we converted it to an array

In [ ]:
y=df['encode_label']

## <font color='#0F52BA'> Öznitelik Mühendisliği - Feature Engineering

### <font color=#007fff> Model - Modelling

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=42)
# TR = modelimizi eğittik.
# EN = We trained our model.

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)

In [ ]:
model = Sequential()
# TR = Ardışık bir model oluşturur, katmanlar sıralı olarak eklenir.
# EN = Creates a sequential model where layers are added in a linear stack.

model.add(InputLayer(input_shape=(size,size,3)))
# TR = Modelin giriş katmanını tanımlar ve veri boyutunu belirtir.
# EN = Defines the input layer of the model and specifies the data dimensions.

model.add(Reshape(target_shape=(size,size,3)))
# TR = Veriyi sizexsizex3 şeklinde yeniden düzenler, gri tonlama görüntüleri için tek bir renk kanalını belirtir.
# EN = Reshapes the data to sizexsizex3, specifying a single color channel for grayscale images.

model.add(Conv2D(filters=12, kernel_size=(3,3), activation='relu'))
# TR = Konvolüsyon işlemi ile özellikleri çıkarır, filtreler görüntüdeki desenleri öğrenir ve 'relu' aktivasyon fonksiyonu ile doğrusal olmayan ilişkileri modellemesini sağlar.
# EN = Extracts features via convolution, where filters learn patterns in the image and the 'relu' activation function introduces non-linearity.

model.add(MaxPooling2D(pool_size=(2,2)))
# TR = Özellik haritasını küçültür ve en belirgin özellikleri seçer, böylece hesaplama maliyeti azalır ve modelin genelleştirme yeteneği artar.
# EN = Reduces the feature map size and selects the most prominent features, reducing computation and improving the model’s ability to generalize.

model.add(Flatten())
# TR = Çok boyutlu veri kümesini tek boyutlu bir vektöre dönüştürür, tam bağlantılı katmanlarla işlem için uygun hale getirir.
# EN = Converts the multi-dimensional feature map into a one-dimensional vector to be processed by fully connected layers.

model.add(Dense(5))
# TR = Sonuçları 5 farklı sınıfa dönüştüren bir tam bağlantılı katman ekler, bu da modelin sınıflandırma yapmasını sağlar.
# EN = Adds a fully connected layer with 10 units to output results for 10 different classes, enabling classification.

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# TR = Modeli 'adam' optimizasyonu ile derler, bu optimizasyon algoritması öğrenmeyi iyileştirir, ve 'SparseCategoricalCrossentropy' kayıp fonksiyonunu kullanarak çok sınıflı sınıflandırma yapar, doğruluk metriğiyle model performansını değerlendirir.
# EN = Compiles the model with the 'adam' optimizer, which improves learning, uses 'SparseCategoricalCrossentropy' loss function for multi-class classification, and evaluates performance with accuracy metric.

history=model.fit(x_train, y_train, validation_split=.10, epochs=10)
# TR = Modeli eğitim verileri (train_images) ve etiketleri (train_labels) ile eğitir, %10'luk bir doğrulama seti kullanır ve 10 dönem boyunca eğitir.
# EN = Trains the model with the training data (train_images) and labels (train_labels), uses 10% of the data for validation, and trains for 10 epochs.

In [ ]:
model.summary()

In [ ]:
loss, _accuracy = model.evaluate(x_test, y_test)
# TR = Test verileri (test_images) ve etiketleri (test_labels) ile modelin performansını değerlendirir, kayıp ve doğruluk değerlerini döndürür.
# EN = Evaluates the model's performance using test data (test_images) and labels (test_labels), returning loss and accuracy values.

In [ ]:
plt.plot(history.history['accuracy'],label='Accuracy')
plt.plot(history.history['val_accuracy'],label='Val_Accuracy')
plt.legend();